In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_core.embeddings import Embeddings
from FlagEmbedding import BGEM3FlagModel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.runnables import RunnableLambda
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableMap

In [2]:
env_path = "/Users/kumarpersonal/Downloads/Inquiry-Assistant/venv/.env"
load_dotenv(dotenv_path=env_path)

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'inquiry-assistant'
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")
print(f"Using Pinecone index: {pinecone_index_name}, {type(pinecone_index_name)}")

groq_api_key = os.getenv("GROQ_API_KEY")

Using Pinecone index: dense-retrieval-index, <class 'str'>


In [3]:
corpus_path = "/Users/kumarpersonal/Downloads/Inquiry-Assistant/Context/corpus.txt"
with open(corpus_path, "r", encoding="utf-8") as f:
    full_text = f.read()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
)
documents = splitter.create_documents([full_text])

for doc in documents:
    doc.metadata["chunk_text"] = doc.page_content

In [4]:
hf_encoder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
class BGEEmbedding(Embeddings):
    def __init__(self):
        self.model = BGEM3FlagModel('BAAI/bge-small-en', use_fp16=False)

    def embed_documents(self, texts):
        texts = ["passage: " + t for t in texts]
        output = self.model.encode(texts)
        dense_vecs = output["dense_vecs"]
        dense_vecs = dense_vecs / np.linalg.norm(dense_vecs, axis=1, keepdims=True)
        return dense_vecs.tolist() 

    def embed_query(self, text):
        output = self.model.encode(["query: " + text])
        dense_vec = output["dense_vecs"][0]
        dense_vec = dense_vec / np.linalg.norm(dense_vec)
        return dense_vec.tolist()
    
embed_func = BGEEmbedding()

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

In [6]:
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(pinecone_index_name)

In [7]:
# vectors_to_upsert = []
# for i, (doc, dense, sparse) in enumerate(zip(documents, dense_vectors, sparse_vectors)):
#     vector = {
#         "id": f"doc-{i}",
#         "values": dense,
#         "sparse_values": sparse,
#         "metadata": doc.metadata
#     }
#     vectors_to_upsert.append(vector)

# index.upsert(vectors=vectors_to_upsert, namespace="default")

# print(f"Upserted {len(vectors_to_upsert)} hybrid vectors to Pinecone index.")

In [8]:
vectorstore = PineconeVectorStore(
    index=index,
    embedding=hf_encoder
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
# retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5, 'fetch_k': 50, 'lambda_mult': 0.5})

In [9]:
llm = ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")

compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever,
    base_compressor=compressor
)

In [10]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an intelligent assistant for Scaler Academy, trained on internal documents, placement records, program curricula, and student feedback. "
     "You should answer questions only based on the context provided. "
     "If the answer is not found in the context, reply with: "
     "“I'm sorry, I couldn't find that information in the available documents.” "
     "Be precise, concise, and maintain a professional and helpful tone."
    ),
    ("human", 
     "Context:\n{context}\n\nQuestion:\n{question}")
])

output_parser = StrOutputParser()

In [15]:
retriever_runnable = RunnableLambda(lambda x: retriever.invoke(x["question"]))

In [17]:
def format_inputs(inputs):
    return {
        "context": "\n\n".join([doc.page_content for doc in inputs["documents"]]),
        "question": inputs["question"]
    }

In [18]:
rag_chain = (
    RunnableMap({
        "documents": retriever_runnable,
        "question": lambda x: x["question"]
    })
    | format_inputs | chat_prompt | llm | output_parser
)

In [19]:
query_text = input("Enter your question: ")
print("\nQuestion:", query_text)

response = rag_chain.invoke({"question": query_text})
print("\nAnswer:", response)


Question: Scaler's placement records

Answer: I'm sorry, I couldn't find specific placement records in the available documents.  

